# Q-Learning with OpenAI Taxi v3

This notebook was created thanks to a course on Deep Reinforcement Learning created by Thomas Simonini. You can find the syllabus here: https://simoninithomas.github.io/Deep_reinforcement_learning_Course/

$$G_{t}=\sum_{k=0}^{\infty} \gamma^{k} R_{t+k+1} w h e r e  \gamma \in[0,1)$$

The formula above represents the **discounted cumulative expected rewards**. It's the sum of the rewards given by the environment at each time step $t$ discounted by $\gamma$. This is a **discount rate** (between 0 and 1) used to increase the effect of **long-term rewards** in earlier time steps and then, as the end of the game approaches and future rewards are less likely to happen, gets smaller and smaller to prioritize **short-term** rewards. This happens because $\gamma$ is elevated to the current time step $k$ and therefore it will be become smaller and smaller with higher $k$.

Another important concept is the **exploration/exploitation** tradeoff. At the start of the training the Q-table is going to be completely useless since it is going to be full of arbitrary values (usually 0s) that weren't the result of the agent playing. Before it actually makes sense to follow the Q-table when deciding on action is is better to have the agent focus on **exploration**, that is making random actions for the sake of exploring the state of game and start filling up the Q-table according to the rewards received.

In the code below this is represented by the **epsilon** variable. This variable gets smaller each **episode** (that is, each game played by the agent) so that there will be a strong focus on random moves at the **start** of the training so that the Q-table can be populated and then, as the training progresses, **epsilon** will get smaller and more and more actions of the agent will be decided based on highest Q-table value for that specific game state.

In this notebook I will implement an agent that can play the OpenAI Gym environment Taxi-v3. The goal of the game is to pick up the passenger at one location and drop him off in another. There are 4 different location marked by 4 different letters. The point system for this environment works as follows:

- You receive +20 points for a successful dropoff
- Lose 1 point for every timestep it takes.
- There is also a 10 point penalty for illegal pick-up and drop-off actions (if you don't drop the passenger in one of the 3 other locations)

The libraries needed are:
- Numpy to generate the Qtable
- Gym for the Taxi environment
- Random to generate random numbers

In [13]:
import numpy as np
import gym
import random

In [14]:
# Creating Taxi environment and rendering an example game state
env = gym.make("Taxi-v3")
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [15]:
# Obtaining number of possible actions (number of rows of Q-table) and
# number of possible states (number of columns)
action_size = env.action_space.n
print("Action size ", action_size)

state_size = env.observation_space.n
print("State size ", state_size)

Action size  6
State size  500


In [16]:
# Creating empty Q-table
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [17]:
# Setting hyperparameters
total_episodes = 50000 # Total train episodes
total_test_episodes = 1000 
max_steps = 99

learning_rate = 0.7
gamma = 0.618

# setting exploration parameters
epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.01

In [18]:
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    
    # Executing up to 99 actions
    for step in range(max_steps):
        # Obtaining random integer.
        exp_exp_tradeoff = random.uniform(0,1)
        # If tradeoff > epsilon the action will be based on the biggest Q-value for this state
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
        # else the action will be random (exploring the environment)
        else:
            action = env.action_space.sample()
        
        # Passing the action into the step method of the environment
        new_state, reward, done, info = env.step(action)
        
        # Updating the Q-table with new value based on the results of the last action
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma *
                                    np.max(qtable[new_state, :]) - qtable[state, action])
        
        # Overwriting old state to use new_state of the environment in next loop
        state = new_state
        
        # Break the loop if the agent finishes the game
        if done:
            break
            
    # Updating epsilon to decrease the ratio of exploration/exploitation over time
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)

In [19]:
env.reset()
rewards = []

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0

    for step in range(max_steps):
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        total_rewards += reward
        
        if done:
            rewards.append(total_rewards)
            break
        state = new_state
env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

Score over time: 7.922


Given the need to try out different parameters I have created two functions in the `qlearning_functions.py` file to test various hyperparameters and compare the scores. This will mostly come down to playing out with different values

In [20]:
from qlearning_functions import agent_testing, agent_training

Testing learning_rate values of 0.1, 0.4, 0.7 and 1.

In [21]:
trained_qtables = [agent_training(env, qtable, learning_rate=learning_rate) 
                   for learning_rate in [0.1, 0.4, 0.7, 1]]

In [22]:
scores = [agent_testing(env, qtable=trained_table) for trained_table in trained_qtables]

In [23]:
scores

['7.905', '7.906', '7.876', '7.865']

Testing gamma values of 0.4, 0.5, 0.6 and 0.7 together with the learning rate associated with the highest score, 0.4.

In [24]:
trained_qtables = [agent_training(env, qtable, learning_rate=0.4, gamma=gamma) 
                   for gamma in [0.4, 0.5, 0.6, 0.7]]

In [25]:
scores = [agent_testing(env, qtable=trained_table) for trained_table in trained_qtables]

In [26]:
scores

['7.504', '7.945', '8.023', '2.424']

Testing higher epsilon decay rates of 0.03, 0.07, 0.1 and 0.15.

In [27]:
trained_qtables = [agent_training(env, qtable, learning_rate=0.4, gamma=0.6, decay_rate=decay_rate) 
                   for decay_rate in [0.03, 0.07, 0.1, 0.15]]

In [28]:
scores = [agent_testing(env, qtable=trained_table) for trained_table in trained_qtables]

In [29]:
scores

['7.823', '7.868', '7.967', '7.957']